In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Subset

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                      download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform)

trainset_sub = Subset(trainset, indices=range(3200))
testset_sub = Subset(testset, indices=range(600))

trainloader = torch.utils.data.DataLoader(trainset_sub, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset_sub, batch_size=4,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
class model_CIFAR10(nn.Module):
    def __init__(self):
        super(model_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(6 * 6 * 32, 64)
        self.fc2 = nn.Linear(64, 10)
        #self.fc3 = nn.Linear(32, 10)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        #print(x.shape)
        x = self.pool2(x)
        print(x.shape)
        #x = self.pool2(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], 6 * 6 * 32)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x
    
    def train(self, batches):
        losses = []
        for epoch in range(3):
            running_loss = 0.0
            for i, batch in enumerate(batches):
                X_batch, y_batch = batch   

                self.optimizer.zero_grad()
            
                y_pred = self.forward(X_batch)
                loss = self.loss_fn(y_pred, y_batch)
                loss.backward()
                self.optimizer.step()
            
                running_loss += loss.item()
            
                if i % 200 == 199:
                    print('{} loss: {}'.format(i + 1 + epoch * 800, 
                                               round(running_loss / 200, 
                                                     3)))
                    losses.append(running_loss)
                    running_loss = 0.0 
                
        plt.plot(np.arange(len(losses)), losses)
        plt.show()
        print('train ended')
        
    def test(self, data):
        class_correct = list(0 for i in range(10))
        class_total = list(0 for i in range(10))
        classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        
        with torch.no_grad():
            for i, batch in enumerate(data):
                images, labels = batch
                y_pred = self.forward(images)
                _, predicted = torch.max(y_pred, 1)
                
                checker = (predicted.detach() == labels)
                
                for i in range(4):
                    label = labels[i]
                    class_correct[label] += checker[i].item()
                    class_total[label] += 1
                    
        for idx, name_class in enumerate(classes):
            print('Accuracy of {}: {}'.format(name_class, round(class_correct[idx] / class_total[idx] * 100, 3)))
        
        print('Mean accuracy = {}'.format(sum(class_correct) / sum(class_total) * 100))

In [14]:
class model_CIFAR10_maxPool(model_CIFAR10):
    def __init__(self):
        super(model_CIFAR10_maxPool, self).__init__()
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.pool2 = torch.nn.MaxPool2d(2, 2)

In [15]:
m2_max = model_CIFAR10_maxPool()
m2_max.train(trainloader)

torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([4, 32, 6, 6])
torch.Size([

KeyboardInterrupt: 

In [31]:
m2_max.test(testloader)

Accuracy of plane: 28.788
Accuracy of car: 23.913
Accuracy of bird: 29.31
Accuracy of cat: 3.571
Accuracy of deer: 1.961
Accuracy of dog: 40.323
Accuracy of frog: 71.429
Accuracy of horse: 7.018
Accuracy of ship: 61.905
Accuracy of truck: 46.479
Mean accuracy = 33.5
